In [1]:
import numpy as np
from matplotlib import pyplot as plt

# Dynamic Expectation Maximisation

## By André van Schaik

### __[International Centre for Neuromorphic Systems](https://westernsydney.edu.au/icns)__        10/01/2019

I made this notebook on Dynamic Expectation Maximisation (DEM) following "Hierarchical Models in the Brain", by Karl Friston, PLoS Computational Biology 4(11): e1000211. [doi:10.1371/journal.pcbi.1000211](https://doi.org/10.1371/journal.pcbi.1000211). <span style="color:red">This is very much a (slow) work in progress and as yet unfinished. It's on GitHub for my own convenience and not ready for public consumption. Read at your own peril!</span>

### Dynamic model ###

We can write a dynamic input-state-output model as:

\begin{align*}
\tilde y &= \tilde g + \tilde z \\
D \tilde x &= \tilde f + \tilde w \tag{1}
\end{align*}

where $\tilde y$ are the observations, and $\tilde x$ are the hidden states of the system. The $\tilde a$ notation indicates variables and functions in generalised coordinates of motion $\tilde a = [a, a', a'', a''', ...]^T$. Here, $a'$ is the *value* of the derivative of $a$ with respect to time; in other words, it is a dimensionless number, even though the time derivative obviously has a unit of $[s^{-1}]$. One way of interpreting this is that $a' = \tau\,da/dt$ where $\tau = dt$ and all time is measured in units of $dt$, and similarly for the higher orders of motion.

$D$ is a block-matrix derivative operator, whose first leading-diagonal contains identity matrices. This operator simply shifts the vectors of generalised motion so $a[i]$ that is replaced by $a[i+1]$.

The predicted sensor response $\tilde g$ and motion $\tilde f$ of the hidden states $\tilde x$ in absence of random fluctuations are:

\begin{align*}
\begin{split}
g &= g(x, \nu) \\
g' &= g_x x' + g_\nu \nu' \\
g'' &= g_x x'' + g_\nu \nu'' \\
&\phantom{g=\,} \vdots \\
\end{split}
\:\:\:
\begin{split}
f &= f(x, \nu) \\
f' &= f_x x' + f_\nu \nu' \\
f'' &= f_x x'' + f_\nu \nu'' \\
&\phantom{f=\,} \vdots \\
\end{split}
\end{align*}

Here, $f$ and $g$ are continuous nonlinear functions and $\tilde \nu$ are known causes or inputs, which can also result from actions by the agent. The notation $a_b$ is shorthand for $\partial{a}/\partial{b}$. We assume that the observation noise $\tilde z$ follows a zero-mean Gaussian distribution $p(\tilde z) = \mathcal{N}(0, \tilde \Sigma^z)$ and similarly for the state noise $p(\tilde w) = \mathcal{N}(0, \tilde \Sigma^w)$. The input drive is also Gaussian but with a mean that can be different from zero: $p(\tilde \nu) = N(\tilde \eta^\nu, \tilde C^\nu)$, where we use $\tilde C^\nu$ instead of $\tilde \Sigma^\nu$ to indicate this is a *prior* covariance, rather than a *conditional* covariance. We can then evaluate the joint density over observations $\tilde y$, hidden states $\tilde x$, and inputs $\tilde \nu$:

\begin{align*}
p(\tilde y, \tilde x, \tilde \nu \,|\, \theta, \lambda) &= p(\tilde y \,|\, \tilde x, \tilde \nu, \theta, \lambda) \; p(\tilde x \,|\, \tilde \nu, \theta, \lambda) \; p(\tilde \nu) \\
p(\tilde y \,|\, \tilde x, \tilde \nu, \theta, \lambda) &= \mathcal{N}(\tilde y : \tilde g, \tilde \Sigma(\lambda)^z) \\
p(\tilde x \,|\, \tilde \nu, \theta, \lambda) &= \mathcal{N}(D\tilde x : \tilde f, \tilde \Sigma(\lambda)^w) \\
p(\tilde \nu) &= \mathcal{N}(\tilde \nu : \eta^\nu, C^\nu)
\end{align*}

where $\theta$ contains the parameters describing $f$ and $g$, and $\lambda$ are hyperparameters which control the amplitude and smoothness of the random fluctuations. Here we have indicated explicitly which random variable is generated by each normal distribution. According to $(1)$, the random variable for state transition is $D\tilde x$, which therefore links different levels of motion.

Finally, we also assume Gaussian priors for the hyperparameters $\lambda$ and $\theta$:

\begin{align*}
p(\lambda) &= \mathcal{N}(\lambda : \eta^\lambda, C^\lambda)\\
p(\theta) &= \mathcal{N}(\theta : \eta^\theta, C^\theta)
\end{align*}

This allows us to write the directed Bayesian graph for the model:

<img src="DM.png" width="600">

### Model inversion ###

For model inversion, we are trying to estimate the parameters $\vartheta$ of a model given some observations $y$ and a model $m$ by maximising the conditional density $p(\vartheta \,|\, \tilde y, m)$. However, this density is in general not directly calculable as it involves normalising over all possible observations. *Variational Bayes* suggests a workaround by minimising the Kullback-Leibler divergence between what it believes the state of its environment is (encoded in a Recognition density $q(\theta)$) and the true Bayesian posterior.

\begin{align*}
D_{KL}(\: q(\vartheta) \; || \; p(\vartheta \,|\, \tilde y, m) \: ) = \int{q(\vartheta) \: \ln \frac{q(\vartheta)}{p(\vartheta\,|\, \tilde y, m)} \: d\vartheta}
\end{align*}

The KL divergence is a measure of the difference between two probability distributions, is always positve, and is 0 if and only if the two distributions are the same. Thus adapting $q(\vartheta)$ to minimise this KL divergence will result in $q(\vartheta)$ being a close approximation of $p(\vartheta\,|\, \tilde y, m)$. 

Obviously, to evaluate this KL divergence directly, we would still need to be able to calculate $p(\vartheta\,|\, \tilde y, m)$ and we seem to have made no progress. However, the FEP uses the fact that $p(\vartheta, \tilde y, m) = p(\vartheta\,|\, \tilde y, m) p(\tilde y, m)$, to write this as:

\begin{align*}
D_{KL}(\: q(\vartheta) \; || \; p(\vartheta\,|\, \tilde y, m) \: ) &= \int{q(\vartheta) \: \ln \frac{q(\vartheta)}{p(\vartheta, \tilde y, m)/p(\tilde y, m)} \: d\vartheta} \\
&= \int{q(\vartheta) \: \{ \ln\:q(\vartheta) - \ln\:p(\vartheta, \tilde y, m) + \ln\:p(\tilde y, m) \} \: d\vartheta} \\
&= \int{q(\vartheta) \: \ln \frac{q(\vartheta)}{p(\vartheta, \tilde y, m)} \: d\vartheta} + \int{q(\vartheta) \: \ln p(\tilde y, m) \: d\vartheta} \\
&= \int{q(\vartheta) \: \ln \frac{q(\vartheta)}{p(\vartheta, \tilde y, m)} \: d\vartheta} + \ln p(\tilde y, m) \int{q(\vartheta) \: d\vartheta} \\
&= \int{q(\vartheta) \: \ln \frac{q(\vartheta)}{p(\vartheta, \tilde y, m)} \: d\vartheta} + \ln p(\tilde y, m) \\
\end{align*}

since $\int{q(\vartheta) \: d\vartheta} = 1$ by definition of a probability density. We continue by writing:

\begin{align*}
D_{KL}(\: q(\vartheta) \; || \; p(\vartheta\,|\, \tilde y, m) \: ) &= \ln p(\tilde y, m) - F\\
F &= -\int{q(\vartheta) \: \ln \frac{q(\vartheta)}{p(\vartheta, \tilde y, m)} \: d\vartheta} \\
&= -D_{KL}(\: q(\vartheta) \; || \; p(\vartheta, \tilde y, m) \: )\\
\end{align*}

The joint density $p(\vartheta,\tilde y, m)$ is called the generative density, and represents the agent's belief in how the world works. It can be factorised into $p(\vartheta,\tilde y, m) = p(\tilde y, \vartheta, m) = p(\tilde y \,|\, \vartheta, m)\:p(\vartheta, m)$ where a prior $p(\vartheta, m)$ encodes the agent's beliefs for the world states prior to new sensory input, and a likelihood $p(\tilde y|\vartheta, m)$ encodes how the agent's sensory signals relate to the world states. Thus, if we have a model for how the world states generate sensory perception (or if we can learn one), we can calculate $F$, which is called the *Variational Free Energy*, and is the negative of the KL divergence between the Recognition density, $q(\vartheta)$, and the Generative density, $p(\vartheta, \tilde y, m)$. We probably don't know $p(\tilde y, m)$, but, since this doesn't depend on $\vartheta$, it plays no role in optimising $q(\vartheta)$. We can simply maximise $F$ to make $q(\vartheta)$ the best possible approximation of $p(\vartheta,\tilde y, m)$, and thereby maximise $p(\vartheta\,|\, \tilde y, m)$.

Given a model, we can determine the probability of the observations under the model by:

\begin{align*}
\ln p(\tilde y, m) &= \ln p(\tilde y \,|\, m) + \ln p(m) \\
\ln p(\tilde y \,|\, m) &= \ln p(\tilde y, m) - \ln p(m) \\
&= F + D_{KL}(\: q(\vartheta) \; || \; p(\vartheta\,|\, \tilde y, m) \: ) - \ln p(m)
\end{align*}

Thus for a given model ($p(m)=1$), we can write the log likelihood as:

\begin{align*}
\ln p(\tilde y \,|\, m) &= F + D_{KL}(\: q(\vartheta) \; || \; p(\vartheta\,|\, \tilde y, m) \: )
\end{align*}

This indicates that $F$ can be used as a lower-bound for the log-evidence, since the KL divergence term is always positive and is $0$ if and only if $q(\vartheta) =  p(\vartheta\,|\, \tilde y, m)$.

$F$ can also be expressed as:

\begin{align*}
F &= -\int{q(\vartheta) \: \ln \frac{q(\vartheta)}{p(\vartheta, \tilde y, m)} \: d\vartheta} \\
&= \left< \ln p(\vartheta, \tilde y, m) \right>_q - \left< \ln q(\vartheta) \right>_q
\end{align*}

which comprises the internal energy $U(\vartheta, \tilde y) = \ln p(\vartheta, \tilde y)$ of a given model $m$ expected under $q(\vartheta)$ and the entropy of $q(\vartheta)$, which is a measure of its uncertainty.



### Optimisation ###

The introduction of $q(\vartheta)$ converts the difficult integration problem inherent in Bayesian Inference into a much simpler optimisation problem of adapting $q(\vartheta)$ to maximise $F$. To further simplify calculation, we usually assume that the model parameters can be partitioned over the states $u = [\tilde \nu, \tilde x]^T$, the parameters $\theta$, and the hyperparameters $\lambda$, as:

\begin{align*}
q(\vartheta) &= q(u(t)) \, q(\theta) \, q(\lambda) \\
&= \prod_i q(\vartheta^i) \\
\vartheta^i &= \{u(t), \theta, \lambda\}
\end{align*}

This partition is called the *mean field* approximation in statistical physics. We further assume that over the timescale of inference, only the states $u$ change with time $t$, while the (hyper)parameters are assumed constant.

Under this partition, optimisation is still achieved by maximising the Free Energy, but we can now do this separately for each partition, by averaging over the other partitions. To show this, we define $F$ as an integral over the parameter partitions:

\begin{align*}
F &= \int f^i \, d\vartheta^i \\
&= \int{q(\vartheta) \: \ln p(\vartheta, \tilde y, m) \: d\vartheta} - \int{q(\vartheta) \: \ln q(\vartheta) \: d\vartheta} \\
&= \iint{q(\vartheta^i) \: q(\vartheta^{\backslash i}) \: \ln p(\vartheta, \tilde y, m) \: d\vartheta^{\backslash i} \: d\vartheta^i} - \iint{q(\vartheta^i) \: q(\vartheta^{\backslash i}) \: \ln q(\vartheta) \: d\vartheta^{\backslash i} \: d\vartheta^i} \\
&= \int{\left( \int{q(\vartheta^i) \: q(\vartheta^{\backslash i}) \: \ln p(\vartheta, \tilde y, m) \: d\vartheta^{\backslash i} } - \int{q(\vartheta^i) \: q(\vartheta^{\backslash i}) \: \ln q(\vartheta) \: d\vartheta^{\backslash i} } \right) \: d\vartheta^i }\\
f^i &= \int{q(\vartheta^i) \: q(\vartheta^{\backslash i}) \: \ln p(\vartheta, \tilde y, m) \: d\vartheta^{\backslash i} } - \int{q(\vartheta^i) \: q(\vartheta^{\backslash i}) \: \ln q(\vartheta) \: d\vartheta^{\backslash i}} \\
&= q(\vartheta^i) \: \int{ q(\vartheta^{\backslash i}) \: U(\vartheta, \tilde y) \: d\vartheta^{\backslash i} } - q(\vartheta^i) \: \int{q(\vartheta^{\backslash i}) \: (\ln q(\vartheta^i) + \ln q(\vartheta^{\backslash i})) \: d\vartheta^{\backslash i}} \\
&= q(\vartheta^i) \left( V(\vartheta^i) - \ln q(\vartheta^i) - \int{q(\vartheta^{\backslash i}) \: \ln q(\vartheta^{\backslash i}) \: d\vartheta^{\backslash i}} \right) \\
F &= \int q(\vartheta^i) \: \left( V(\vartheta^i) - \ln q(\vartheta^i) - Z^i \right)\, d\vartheta^i \\
\end{align*}

Here, $\vartheta^{\backslash i}$ denotes all parameters not in set $i$, i.e., its Markov blanket,  $Z^i$ contains all the terms of $f^i$ that do not depend on $\vartheta^i$, and

\begin{align*}
V(\vartheta^i) &= \int{q(\vartheta^{\backslash i}) \: \ln p(\vartheta, \tilde y, m) \: d\vartheta^{\backslash i} } = \int{ q(\vartheta^{\backslash i}) \: U(\vartheta, \tilde y) \: d\vartheta^{\backslash i} } = \left< U(\vartheta) \right>_{q(\vartheta^{\backslash i})}\\
\end{align*}

The free energy is maximised when the derivative of $F$ with respect to the distribution $q(\vartheta^i)$ = 0. The Fundamental Lemma of variational calculus states:

\begin{align*}
\delta_{q(\vartheta^i)} F &= 0 \Leftrightarrow \left( V(\vartheta^i) - \ln q(\vartheta^i) - Z^i \right) = 0 \\
\ln q(\vartheta^i) &= V(\vartheta^i) - \ln Z^i\\
q(\vartheta^i) &= \frac{1}{Z^i} \exp \left(V(\vartheta^i)\right) = \frac{1}{Z^i} \exp\left(\left< U(\vartheta) \right>_{q(\vartheta^{\backslash i})}\right) 
\end{align*}

Thus, $Z^i$ is a normalisation constant ensuring the distribtion integrates to $1$, and is also called a partition function in physics. The final equation indicates that the variational density over one parameter set is an exponential function of the internal energy averaged over all other parameters.

Given our partitions above, we can then write:

\begin{align*}
q(u(t)) &\propto \exp \left(V(t)\right) \\
V(t) &= \left< U(t) \right>_{q(\theta)q(\lambda)} \\
q(\theta) &\propto \exp \left(\bar{V}^\theta \right) \\
\bar{V}^\theta &= \int \left< U(t) \right>_{q(u)q(\lambda)} \:dt + U^\theta \\
q(\lambda) &\propto \exp \left(\bar{V}^\lambda \right) \\
\bar{V}^\lambda &= \int \left< U(t) \right>_{q(u)q(\theta)} \:dt + U^\lambda \\
\bar{U} &= \int U(t)\:dt + U^\theta + U^\lambda \\
\end{align*}

In a dynamical system, the instantaneous internal energy $U(t)$ is a function of time. Because the parameters and hyperparameters are considered constant over a period of observation, their variational densities are functions of the path integal of this internal energy, which is called *action* in physics. We use the notation $\bar{V}$ and $\bar{U}$ to indicate these integrals. $U^\theta = \ln p(\theta)$ and $U^\lambda = \ln p(\lambda)$ are the prior energies of the parameters and hyperparameters, respectively. 

From these equations we see that the variational density over states can be determined from the instantaneous internal energy averaged over parameters and hyperparameters, whereas the density over parameters and hyperparameters can only be determined when data has been observed over a certain amount of time. In the absence of data, the integrals will be zero, and the conditional density simply reduces to the prior density.

*Variational Bayes* assumes the above equations are analytically tractable, which needs needs the choice of appropriate (conjugate) priors. The conditional distributions $q(\vartheta^i)$ above can then be updated through iteration as new data becomes available:

\begin{align*}
U(t) &= \ln p(\vartheta, \tilde y, m)(t) \\
\ln q(u(t)) &\propto \left< U(t) \right>_{q(\theta)q(\lambda)} \\
\ln q(\theta) &\propto \int \left< U(t) \right>_{q(u)q(\lambda)} \:dt + \ln p(\theta) \\
\ln q(\lambda) &\propto \int \left< U(t) \right>_{q(u)q(\theta)} \:dt + \ln p(\lambda)
\end{align*}



### The Laplace approximation ###

If the above equations are not analytically tractable, or if we want to avoid those calculations, we can apply the Laplace approximation. The Laplace approximation assumes that the marginals of the conditional density assume a Gaussian form, i.e., $q(\vartheta^i) = \mathcal{N}(\vartheta^i : \mu^i, \Sigma^i)$, where $\mu^i$ and $\Sigma^i$ are the sufficient statistics. For notational clarity, we will use $\mu^i$,  $\Sigma^i$, and $\Pi^i$ for the conditional mean, covariance, and precision of the $i^\text{th}$ marginal, respectively, and $\eta^i$,  $C^i$, and $P^i$ for their priors. This approximation simplifies the updates to the marginals of the conditional densities.

For each partition $\vartheta^i$, we can then write:

\begin{align*}
q(\vartheta^i) &= \frac{1}{\sqrt{2\pi |\Sigma^i|}} \exp \left( \frac{-(\vartheta^i - \mu^i)^2}{2\Sigma^i} \right) \\
&= \frac{1}{Z^i} \exp \left( -\varepsilon(\vartheta^i) \right) \\ 
Z^i &= \sqrt{(2\pi)^{n^i} |\Sigma^i|} \\
\varepsilon(\vartheta^i) &= \frac{(\vartheta^i - \mu^i)^2}{2\Sigma^i} \\
&= \frac{1}{2} (\vartheta^i - \mu^i)^T {\Sigma^i}^{-1} (\vartheta^i - \mu^i)
\end{align*}

Where $n^i$ is the number of parameters in partition $i$.

Recall that the Free Energy was defined as:

\begin{align*}
F &= -\int{q(\vartheta) \: \ln \frac{q(\vartheta)}{p(\vartheta, \tilde y, m)} \: d\vartheta} \\
&= - \int{q(\vartheta) \: \ln q(\vartheta) \: d\vartheta} + \int{q(\vartheta) \: \ln p(\vartheta, \tilde y, m) \: d\vartheta} \\
&= - \int{q(\vartheta) \: \ln \prod_i q(\vartheta^i) \: d\vartheta} + \left< U \right>_q \\
&= \int{q(\vartheta) \: \sum_i(\ln Z^i + \varepsilon(\vartheta^i)) \: d\vartheta} + \left< U \right>_q \\
&= \sum_i(\ln Z^i)\int{q(\vartheta) \: d\vartheta} + \int{q(\vartheta) \: \sum_i(\varepsilon(\vartheta^i)) \: d\vartheta} + \left< U \right>_q \\
&\left(\int{q(\vartheta) \: d\vartheta} = 1\right) \\
&= \sum_i(\ln Z^i) + \int{q(\vartheta) \: \sum_i(\frac{1}{2} (\vartheta^i - \mu^i)^T {\Sigma^i}^{-1} (\vartheta^i - \mu^i)) \: d\vartheta} + \left< U \right>_q \\
&= \sum_i(\ln Z^i) + \sum_i(\frac{1}{2} \int{q(\vartheta^i) \: (\vartheta^i - \mu^i)^T {\Sigma^i}^{-1} (\vartheta^i - \mu^i) \: d\vartheta^i}) + \left< U \right>_q \\
&\left( \text{using } a^T B a = \text{tr} \left( a a^T B \right) \right)\\
&= \sum_i(\ln Z^i) + \sum_i \frac{1}{2}\int{q(\vartheta^i) \: \text{tr} \left((\vartheta^i - \mu^i) (\vartheta^i - \mu^i)^T  {\Sigma^i}^{-1} \right) \: d\vartheta^i}) + \left< U \right>_q \\
&= \sum_i(\ln Z^i) + \sum_i \frac{1}{2} \int{\text{tr} \left(q(\vartheta^i) \: (\vartheta^i - \mu^i) (\vartheta^i - \mu^i)^T  {\Sigma^i}^{-1} \right) \: d\vartheta^i}) + \left< U \right>_q \\
&= \sum_i(\ln Z^i) + \sum_i \frac{1}{2} \text{tr} \left(\int{q(\vartheta^i) \: (\vartheta^i - \mu^i) (\vartheta^i - \mu^i)^T  {\Sigma^i}^{-1} \: d\vartheta^i}) \right) + \left< U \right>_q \\
&= \sum_i(\ln Z^i) + \sum_i \frac{1}{2} \text{tr} \left(\Sigma^i {\Sigma^i}^{-1} \right) + \left< U \right>_q \\
&= \sum_i(\ln Z^i + \frac{n^i}{2}) + \left< U \right>_q \\
\end{align*}


Now we still need to find an expression we can calculate for $\left< U \right>_q$. To do this, a further approximation assumes that $q$ is sharply peaked at its mean value $\mu$, so that the integration is only non-zero close to $\vartheta = \mu$. We can then use a Taylor expansion around the mean up to second order to obtain: 

\begin{align*}
\left< U \right>_q &= \int{q(\vartheta) \: U(\vartheta, \tilde y) \: d\vartheta} \\
&= \int{q(\vartheta) \: \left\{ U(\mu, \tilde y) + \left[ \frac{dU}{d\vartheta} \right]_\mu \delta\vartheta + \frac{1}{2} \left[ \frac{d^2U}{d\vartheta^2} \right]_\mu \delta\vartheta^2 \right\} \: d\vartheta} \\
&= U(\mu, \tilde y) + \left[ \frac{dU}{d\vartheta} \right]_\mu \int{q(\vartheta) \: (\vartheta - \mu) \: d\vartheta} + \frac{1}{2} \int{ q(\vartheta) \: (\vartheta - \mu)^T \left[ \frac{d^2U}{d\vartheta^2} \right]_\mu (\vartheta - \mu) \: d\vartheta} \\
&= U(\mu, \tilde y) + \left[ \frac{dU}{d\vartheta} \right]_\mu \left\{ \int{\vartheta q(\vartheta) \: d\vartheta} - \mu \right\} + \frac{1}{2} \int{ q(\vartheta) \: \text{tr} \left( \left[ \frac{d^2U}{d\vartheta^2} \right]_\mu (\vartheta - \mu) (\vartheta - \mu)^T \right) \: d\vartheta} \\
&= U(\mu, \tilde y) + \left[ \frac{dU}{d\vartheta} \right]_\mu \left\{ \int{\vartheta q(\vartheta) \: d\vartheta} - \mu \right\} + \frac{1}{2} \text{tr} \left( \left[ \frac{d^2U}{d\vartheta^2} \right]_\mu \int{ q(\vartheta) \: (\vartheta - \mu) (\vartheta - \mu)^T \: d\vartheta} \right) \\
&= U(\mu, \tilde y) + \frac{1}{2} \text{tr} \left( \left[ \frac{d^2U}{d\vartheta^2} \right]_\mu \Sigma \right) \\
\end{align*}

This now allows us to write for the free energy:

\begin{align*}
F &= \sum_i(\ln Z^i + \frac{n^i}{2}) + \left< U \right>_q \\
&= \sum_i(\frac{1}{2} \ln (2\pi)^{n^i} |\Sigma^i| + \frac{n^i}{2})  + U(\mu, \tilde y) + \frac{1}{2} \sum_i \text{tr} \left( \left[ \frac{d^2U}{d{\vartheta^i}^2} \right]_{\mu^i} \Sigma^i \right)\\
&= \sum_i(\frac{n^i}{2} \ln 2\pi e)  + U(\mu, \tilde y) + \frac{1}{2} \sum_i \left\{ \text{tr} \left( \left[ \frac{d^2U}{d{\vartheta^i}^2} \right]_{\mu^i} \Sigma^i \right)+ \ln |\Sigma^i| \right\} \\
\end{align*}

To find the optimal variances, we maximise the free energy with respect to the variances, so that the partial derivatives are zero:

\begin{align*}
\frac{d \text{tr} \left( B A \right)}{dA} &= B^T \\
\frac{d \ln |A|}{dA} &= {{A}^{-1}}^T \\
\frac{dF}{d\Sigma^i} &= \frac{1}{2} \left\{ \left[ \frac{d^2U}{d{\vartheta^i}^2} \right]_{\mu^i} + {\Sigma^i}^{-1} \right\}^T = 0 \\
\Sigma^{i*} &=  - \left[ \frac{d^2U}{d{\vartheta^i}^2} \right]_{\mu^i}^{-1} \\
\end{align*}

where we use the notation $\Sigma^{i*}$ to indicate this is the optimal variance which maximises the free energy.

Finally, this allows us to write for the free energy:

\begin{align*}
F &= \frac{1}{2} \sum_i(\ln (2\pi)^{n_i} |\Sigma^{i*}| + n^i)  + U(\mu, \tilde y) + \frac{1}{2} \sum_i \text{tr}(-I)\\
&= U(\mu, \tilde y) + \frac{1}{2} \sum_i \ln (2\pi)^{n^i} |\Sigma^{i*}| \\
&= U(\mu, \tilde y) + \frac{1}{2} \sum_i \left( \ln |\Sigma^{i*}| + n^i \ln 2\pi \right)
\end{align*}

We now proceed to obtain an expression for the variational distribution of each partition under the Laplace approximation.

\begin{align*}
\ln q(u(t)) &\propto V(t) = \left< U(t) \right>_{q(\theta)q(\lambda)}\\
\ln q(\theta) &\propto \bar{V}^\theta = \int \left< U(t) \right>_{q(u)q(\lambda)} \:dt + U^\theta\\
\ln q(\lambda) &\propto \bar{V}^\lambda = \int \left< U(t) \right>_{q(u)q(\theta)} \:dt + U^\lambda\\
V(t) = \left< U(t) \right>_{q(\theta)q(\lambda)} &= \int q(\lambda) q(\theta) \ln p(\vartheta, \tilde y, m) d\theta d\lambda \\
&= \int q(\lambda) q(\theta) \left( -\frac{1}{2} \left( u(t) - \mu^u \right)^T {\Sigma^u}^{-1} \left( u(t) - \mu^u \right) \right) d\theta d\lambda \\
&+ \int q(\lambda) q(\theta) \left( -\frac{1}{2} \left( \theta - \mu^{\theta} \right)^T {\Sigma^{\theta}}^{-1} \left( \theta - \mu^{\theta} \right) \right) d\theta d\lambda \\
&+ \int q(\lambda) q(\theta) \left( -\frac{1}{2} \left( \lambda - \mu^{\lambda} \right)^T {\Sigma^{\lambda}}^{-1} \left( \lambda - \mu^{\lambda} \right) \right) d\theta d\lambda \\
&= \int q(\lambda) q(\theta) \ln \left( p(u,t|\mu^\theta, \mu^\lambda) \right) d\theta d\lambda \\
&+ \int q(\lambda) q(\theta) \left( \frac{1}{2} \left( \theta - \mu^{\theta} \right)^T \left[ \frac{d^2U(t)}{d{\theta}^2} \right]_{\mu^\theta} \left( \theta - \mu^{\theta} \right) \right) d\theta d\lambda \\
&+ \int q(\lambda) q(\theta) \left( \frac{1}{2} \left( \lambda - \mu^{\lambda} \right)^T \left[ \frac{d^2U(t)}{d{\lambda}^2} \right]_{\mu^\lambda} \left( \lambda - \mu^{\lambda} \right) \right) d\theta d\lambda \\
&\left( \text{and using } a^T B a = \text{tr} \left( a a^T B \right) \right)\\
&= U(u,t|\mu^\theta, \mu^\lambda) \\
&+ \int q(\theta) \text{tr} \left( \frac{1}{2} \left( \theta - \mu^{\theta} \right) \left( \theta - \mu^{\theta} \right)^T \left[ \frac{d^2U(t)}{d{\theta}^2} \right]_{\mu^\theta} \right) d\theta \\
&+ \int q(\lambda) \text{tr} \left( \frac{1}{2} \left( \lambda - \mu^{\lambda} \right) \left( \lambda - \mu^{\lambda} \right)^T \left[ \frac{d^2U(t)}{d{\lambda}^2} \right]_{\mu^\lambda} \right) d\lambda \\
&= U(u,t|\mu^\theta, \mu^\lambda) + \frac{1}{2} \text{tr}\left( \Sigma^\theta \left[ \frac{d^2U(t)}{d{\theta}^2} \right]_{\mu^\theta} \right) + \frac{1}{2} \text{tr} \left(\Sigma^\lambda \left[ \frac{d^2U(t)}{d{\lambda}^2} \right]_{\mu^\lambda} \right) \\
\bar{V}^u &= \int V(t) \:dt \\
&= \int U(u, t|\mu^\theta, \mu^\lambda) + W(t)^\theta + W(t)^\lambda \:dt \\
\text{Similarly, we find:}\\
\bar{V}^\theta &= \int U(\mu^u, t|\theta, \mu^\lambda) + W(t)^u + W(t)^\lambda \:dt + U^\theta \\
\bar{V}^\lambda &= \int U(\mu^u, t|\mu^\theta, \lambda) + W(t)^u + W(t)^\theta \:dt + U^\lambda \\
W(t)^u &= \frac{1}{2} \text{tr}(\Sigma^u U(t)_{uu}) \\
W(t)^\theta &= \frac{1}{2} \text{tr}(\Sigma^\theta U(t)_{\theta\theta}) \\
W(t)^\lambda &= \frac{1}{2} \text{tr}(\Sigma^\lambda U(t)_{\lambda\lambda}) \\
\end{align*}

Also, the conditional precisions are equal to the negative curvatures of the internal action:

\begin{align*}
\bar{U} &= \int U(t)\:dt + U^\theta + U^\lambda \\
\Pi^u &= -\bar{U}_{uu} = -U(t)_{uu} \\
\Pi^\theta &= -\bar{U}_{\theta\theta} = - \int U(t)_{\theta\theta} \: dt - U^\theta_{\theta\theta} \\
\Pi^\lambda &= -\bar{U}_{\lambda\lambda} = - \int U(t)_{\lambda\lambda} \: dt- U^\lambda_{\lambda\lambda} \\
\end{align*}

To get their optimal values, these need to be evaluated at the mode of each partition.

### Temporal smoothness ###

Since the different levels of motion in the generalised coordinates are linked, the actual precision matrix will have off-diagonal elements with non-zero values. The precision is given by the Kronecker product $\tilde \Pi^i = S(\gamma) \otimes \Pi^i$, where $\Pi^i$ is a diagonal matrix specifying the precision of the (often assumed independent) Gaussian noise at each level as determined in the previous section, and $S(\gamma)$ is the temporal precision matrix, which encodes the temporal dependencies between levels, which is a function of their autocorrelations:

\begin{align*}
S =
    \begin{bmatrix}
    1 & 0 & \ddot{\rho}(0) & 0 & \ddot{\ddot{\rho}}(0) & 0 \\
    0 & -\ddot{\rho}(0) & 0 & -\ddot{\ddot{\rho}}(0) & 0 & -\dddot{\dddot{\rho}}(0)\\
    \ddot{\rho}(0) & 0 & \ddot{\ddot{\rho}}(0) & 0 & \dddot{\dddot{\rho}}(0) & 0 \\
    0 & -\ddot{\ddot{\rho}}(0) & 0 & -\dddot{\dddot{\rho}}(0) & 0 & -\ddot{\dddot{\dddot{\rho}}}(0) \\
    \ddot{\ddot{\rho}}(0) & 0 & \dddot{\dddot{\rho}}(0) & 0 & \ddot{\dddot{\dddot{\rho}}}(0) & 0 \\
    0 & -\dddot{\dddot{\rho}}(0) & 0 & -\ddot{\dddot{\dddot{\rho}}}(0) & 0 & -\ddot{\ddot{\dddot{\dddot{\rho}}}}(0)\\
    \end{bmatrix}^{-1}
\end{align*}

(see [here](Generalised%20precision%20matrix.ipynb) for a derivation.) Here $\ddot{\rho}(0)$ is the second derivative of the autocorrelation function evaluated at zero. Note, that because the autocorrelation function is even (symmetrical for positive and negative delays), the odd derivatives of the autocorrelation function are all odd functions, and thus are zero when evaluated at zero.

While $\Sigma^i$ can be evaluated for any analytical autocorrelation function, we assume here that the temporal correlations all have the same Gaussian form, which gives:

\begin{align*}
S &=
    \begin{bmatrix}
    1 & 0 & -\gamma & 0 & 3 \gamma^2 & 0 \\
    0 & \gamma & 0 & -3 \gamma^2 & 0 & 15 \gamma^3 \\
    -\gamma & 0 & 3 \gamma^2 & 0 & -15 \gamma^3 & 0 \\
    0 & -3 \gamma^2 & 0 & 15 \gamma^3 & 0 & -105 \gamma^4 \\
    3 \gamma^2 & 0 & -15 \gamma^3 & 0 & 105 \gamma^4 & 0 \\
    0 & 15 \gamma^3 & 0 & -105 \gamma^4 & 0 & 945 \gamma^5 \\
    \end{bmatrix}^{-1}
\end{align*}

Here, $\gamma$ is the precision parameter of a Gaussian autocorrelation function. Typically, $\gamma > 1$, which ensures the precisions of high-order motion converge quickly to zero. This is important because it enables us to truncate the representation of an infinite number of generalised coordinates to a relatively small number, since high-order prediction errors have a vanishingly small precision. Friston states that an order of n=6 is sufficient in most cases. Also note that my derivation of this matrix compared to Friston has $\gamma_{AvS} = 1/2 \gamma_{KF}$. I believe this is because I took the Gaussian autocorrelation function with precision parameter $\gamma$ to be $\rho(h) = \exp(-\frac{\gamma}{2} h^2)$, whereas Friston's result implies he used $\rho(h) = \exp(-\gamma h^2)$.

Putting all this together, we get:

\begin{align*}
p(\tilde y, \tilde x, \tilde \nu \,|\, \theta, \lambda) &= p(\tilde y \,|\, \tilde x, \tilde \nu, \theta, \lambda) \; p(\tilde x \,|\, \tilde \nu, \theta, \lambda) \; p(\tilde \nu) \\
 &= (2\pi)^{-N_y/2} {|\tilde\Pi^z|}^{1/2} e^{-\frac{1}{2}{\tilde\varepsilon^\nu}^T \tilde\Pi^z \tilde\varepsilon^\nu} (2\pi)^{-N_x/2} {|\tilde\Pi^w|}^{1/2} e^{-\frac{1}{2}{\tilde\varepsilon^x}^T \tilde\Pi^w \tilde\varepsilon^x} \; p(\tilde \nu) \\
 &= (2\pi)^{-(N_y + N_x)/2} (|\tilde\Pi^z| + |\tilde\Pi^w|)^{1/2} e^{-\frac{1}{2}{\tilde\varepsilon^\nu}^T \tilde\Pi^z \tilde\varepsilon^\nu}  e^{-\frac{1}{2}{\tilde\varepsilon^x}^T \tilde\Pi^w \tilde\varepsilon^x} \; p(\tilde \nu)\\
 &= (2\pi)^{-N/2} |\tilde\Pi|^{1/2} e^{-\frac{1}{2}{\tilde\varepsilon}^T \tilde\Pi \tilde\varepsilon} \; p(\tilde \nu)\\ 
\tilde\Pi &= 
    \begin{bmatrix}
    \tilde\Pi^z & \\
    & \tilde\Pi^w
    \end{bmatrix}\\
\tilde\varepsilon &= 
    \begin{bmatrix}
    \tilde\varepsilon^\nu = \tilde y - \tilde g   \\
    \tilde\varepsilon^x = D\tilde x - \tilde f
    \end{bmatrix}\\
N &= \text{Rank}(\tilde\Pi)
\end{align*} 

Here we introduce auxilary variables $\tilde\varepsilon(t)$, which are the prediction errors for the generalised responses and motion of the hidden states, with respective predictions $\tilde g(t)$ and $\tilde f(t)$ and their precisions encoded by $\tilde\Pi$.

The log probability can thus be written:
\begin{align*}
\ln p(\tilde y, \tilde x, \tilde \nu \,|\, \theta, \lambda) &= U(t) =  - \frac{N}{2} \ln 2\pi + \frac{1}{2} \ln |\tilde\Pi| -  \frac{1}{2}{{\tilde\varepsilon}^T \tilde\Pi \tilde\varepsilon} + \ln p(\tilde \nu)
\end{align*}

where the first term is constant, and the fourth term is defined by the input causes and considered known.

### Sequences ###
Our observations are often not in the form of generalised coordinates, but instead as a sequence of observations in time. We can use Taylor's theorem to link the two:

\begin{align*}
y &= [y(1), \dots, y(N)]^T \\
y &= \tilde E(t) \tilde y(t) \\
\tilde E(t) &= E(t) \otimes I \\
E(t)_{ij} &= \frac{(i-t)^{j-1}}{(j-1)!}\\
\tilde y(t) &= \tilde E(t)^{-1} y\\
\end{align*}

For this to work, $E(t)^{-1}$ needs to exist, meaning that the number of elements in the sequence ($i$s) equals the number of levels in the generalised coordinates ($j$s).

### Dynamic Expectation Maximisation ###
Above, we showed that under the Laplace approximation, the optimal precision of each partition can be found by calculating the second derivative (Hessian) of the internal entergy and evaluating it at the mode of that partition. To find the modes we use an optimisation technique. As with conventional variational schemes, we can update the modes of our three partitions in three distinct steps. However, the step dealing with the state (**D**-step) must integrate its conditional mode $\tilde \mu := \mu^u(t)$ over time to accumulate the quantities necessary for updating the parameters (**E**-step) and hyperparameters (**M**-step). We now consider optimising the modes or conditional means in each of these steps.

#### The D-step ####

$\require{color}$In static systems, the mode of the conditional density maximises variational energy, such that $V(t)_u = 0$; this is the solution to a gradient ascent scheme: $\dot{\tilde \mu} = V(t)_u$. In dynamic systems, we also require the path of the mode to be the mode of the path: $\dot{\tilde \mu} = D \tilde \mu$. These two conditions can be satisfied by:

\begin{align*}
\dot{\tilde \mu} - D \tilde \mu = V(t)_u \\
\end{align*}

Here $\dot{\tilde \mu} - D \tilde \mu $ can be regarded as motion in a frame of reference that moves along the trajectory encoded in generalised coordinates. The stationary solution in this moving frame of reference maximises variational action. 

So far, all our discussions have assumed we're operating in continuous time. However, we would execute the D-step at discrete intervals. To apply this we linearise the trajectory following (Ozaki, 1992):

\begin{align*}
\Delta \tilde \mu &= \left( \exp(\Delta t J(t)) - I \right) J(t)^{-1} \dot{\tilde \mu} \\
J(t) &:= \frac{\partial \dot{\tilde \mu}}{\partial u} = \frac{\partial \left(V(t)_u + D \tilde \mu \right)}{\partial u} = V(t)_{uu} + D\\
\end{align*}

Above, for simplicity, we have ignored the dependence of $V(u,t)$ on the observations and input, but we should really inlude all time-varying quantities that affect the variational energy:

\begin{align*}
\begin{bmatrix}
\dot {\tilde y} \\
\dot {\tilde \mu} \\
\dot {\tilde \eta} \\
\end{bmatrix}
&=
\begin{bmatrix}
D \tilde y \\
V(t)_u + D \tilde \mu \\
D \tilde \eta \\
\end{bmatrix} \\
J(t) &= 
\begin{bmatrix}
D & 0& 0 \\
V(t)_{uy} & V(t)_{uu} + D & V(t)_{u\eta}\\
0 & 0 & D \\
\end{bmatrix} \\
U(t) &= - \frac{N}{2} \ln 2\pi + \frac{1}{2} \ln |\tilde\Pi| -  \frac{1}{2}{{\tilde\varepsilon}^T \tilde\Pi \tilde\varepsilon} + \ln p(\tilde \nu) \\
V(t) &= U(t) + W(t)^\theta + W(t)^\lambda \\
V(t)_u &= U(t)_u + W(t)_u^\theta \\
V(t)_{uu} &= U(t)_{uu} + W(t)_{uu}^\theta \\
V(t)_{uy} &= U(t)_{uy}\\
V(t)_{u\eta} &= U(t)_{u\eta} \\
U(t)_u &= - \tilde \varepsilon_u^T \tilde \Pi \tilde \varepsilon \\
U(t)_{uu} &= - \tilde \varepsilon_u^T \tilde \Pi \tilde \varepsilon_u \\
U(t)_{uy} &= -\tilde \varepsilon_u^T \tilde \Pi \tilde \varepsilon_y \\
U(t)_{u\eta} &= -\tilde \varepsilon_u^T \tilde \Pi \tilde \varepsilon_\eta \\
U(t)_{\theta \theta} &= - \tilde \varepsilon_\theta^T \tilde \Pi \tilde \varepsilon_\theta \\
U(t)_{\lambda \lambda} &= - \tilde \varepsilon_\lambda^T \tilde \Pi \tilde \varepsilon_\lambda \\
W(t)^u &= \frac{1}{2} \text{tr}(\Sigma^u U(t)_{uu}) = -\frac{1}{2} \text{tr}(\Sigma^u \tilde \varepsilon_u^T \tilde \Pi \tilde \varepsilon_u) \\
W(t)^\theta &= \frac{1}{2} \text{tr}(\Sigma^\theta U(t)_{\theta\theta}) = -\frac{1}{2} \text{tr}(\Sigma^\theta \tilde \varepsilon_\theta^T \tilde \Pi \tilde \varepsilon_\theta) \\
W(t)^\lambda &= \frac{1}{2} \text{tr}(\Sigma^\lambda U(t)_{\lambda\lambda}) = -\frac{1}{2} \text{tr}(\Sigma^\lambda \tilde \varepsilon_\lambda^T \tilde \Pi \tilde \varepsilon_\lambda) \\
W(t)_{u_i}^\theta &= -\frac{1}{2} \text{tr} \left( \Sigma^\theta \tilde \varepsilon_{\theta u_i}^T \tilde \Pi \tilde \varepsilon_\theta \right) \\
W(t)_{uu_{ij}}^\theta &= -\frac{1}{2} \text{tr} \left( \Sigma^\theta \tilde \varepsilon_{\theta u_i}^T \tilde \Pi \tilde \varepsilon_{\theta u_j} \right) \\
\tilde\varepsilon &= 
    \begin{bmatrix}
    \tilde\varepsilon^\nu = \tilde y - \tilde g   \\
    \tilde\varepsilon^x = D\tilde x - \tilde f
    \end{bmatrix} \\
\tilde\varepsilon_u &= 
    \begin{bmatrix}
    \tilde\varepsilon_\nu^\nu & \tilde\varepsilon_x^\nu \\
    \tilde\varepsilon_\nu^x & \tilde\varepsilon_x^x \\
    \end{bmatrix} =
    \begin{bmatrix}
    g_\nu & g_x \\
    f_\nu & f_x \\
    \end{bmatrix} \\
\tilde \varepsilon_y &=
\begin{bmatrix}
\tilde \varepsilon_y^\nu = I \\
\tilde \varepsilon_y^x = 0 \\
\end{bmatrix} \\
\tilde \varepsilon_\eta &=
\begin{bmatrix}
\tilde \varepsilon_\eta^\nu = I \\
\tilde \varepsilon_\eta^x = 0 \\
\end{bmatrix} \\
\tilde\varepsilon_{u \theta} &= \tilde\varepsilon_{\theta u}^T =
    \begin{bmatrix}
    g_{\nu \theta} & g_{x \theta} \\
    f_{\nu \theta} & f_{x \theta} \\
    \end{bmatrix} \\
\tilde\varepsilon_\theta &= \tilde\varepsilon_{u \theta} \mu^u \\
\Delta \tilde \mu &= \left( \exp(\Delta t J(t)) - I \right) J(t)^{-1} (V(t)_{uu} + D) \\
\Pi^u &= -U(t)_{uu}|_\mu = \{\tilde \varepsilon_u^T \tilde \Pi \tilde \varepsilon_u\}|_\mu \\
\end{align*}

The mean-field term, $W(t)^\lambda$ does not contribute to the D-step because it is not a function of the states. This means uncertainly about the hyperparameters does not affect the update for the states. 

#### The E- and M-steps ####

Exactly the same update procedure can be used for the **E**-step (parameter update) and **M**-step (hyperparameter update). However, in this instance there are no generalised coordinates to consider. Furthermore, we can set the interval between updates to be arbitrarily long because the parameters are updated after the time-series has been integrated. If $\Delta t \to \infty$ is sufficiently large, the matrix exponential in the Ozaki linearisation disappears (because the curvature of the Jacobian is negative definite) giving:

\begin{align*}
\Delta \mu^\theta &= -J(\theta)^{-1} \dot\mu^\theta \\
\dot\mu^\theta & = \bar V_{\theta}^\theta \\
J(\theta) &= \bar V_{\theta \theta}^\theta \\
\Delta \mu^\lambda &= -J(\lambda)^{-1} \dot\mu^\lambda \\
\dot\mu^\lambda & = \bar V_{\lambda}^\lambda \\
J(\lambda) &= \bar V_{\lambda \lambda}^\lambda \\
\end{align*}

This  is a conventional Gauss-Newton update scheme. In this sense, the D-Step can be regarded as a generalization of classical ascent schemes to generalised coordinates that cover dynamic systems. For our HDM, the requisite gradients and curvatures of variational action for the E-step are:

\begin{align*}
U(t) &= - \frac{N}{2} \ln 2\pi + \frac{1}{2} \ln |\tilde\Pi| -  \frac{1}{2}{{\tilde\varepsilon}^T \tilde\Pi \tilde\varepsilon} + \ln p(\tilde \nu) \\
\bar{V}^\theta &= \int U(t) + W(t)^u + W(t)^\lambda \:dt + U^\theta \\
&= \int U(t) + W(t)^u + W(t)^\lambda \:dt + {\varepsilon^\theta}^T \Pi^\theta \varepsilon^\theta - Z^\theta \\
\bar V_\theta^\theta &= \int \tilde \varepsilon_\theta^T \tilde \Pi \tilde \varepsilon + W(t)_\theta^u \:dt - \Pi^\theta \varepsilon^\theta \\
\bar V_{\theta \theta}^\theta &= \int \tilde \varepsilon_\theta^T \tilde \Pi \tilde \varepsilon_\theta + W(t)_{\theta \theta}^u \:dt - \Pi^\theta \\
W(t)_{\theta_i}^u &= -\frac{1}{2} \text{tr} \left( \Sigma_t^u \tilde \varepsilon_{u \theta_i}^T \tilde \Pi \tilde \varepsilon_u \right) \\
W(t)_{\theta_i \theta_j}^u &= -\frac{1}{2} \text{tr} \left( \Sigma_t^u \tilde \varepsilon_{u \theta_i}^T \tilde \Pi \tilde \varepsilon_{u \theta_j} \right) \\
J(\theta) &= \bar V_{\theta \theta}^\theta \\
\Delta \mu^\theta &= -J(\theta)^{-1} \dot\mu^\theta = -{\bar V_{\theta \theta}^\theta}^{-1} \bar V_{\theta}^\theta \\
\Pi^\theta &= -\bar{U}_{\theta\theta}|_{\mu^\theta} = - \int U(t)_{\theta\theta}|_{\mu^\theta} \: dt - U^\theta_{\theta\theta} \\
U(t)_{\theta \theta} &= - \tilde \varepsilon_\theta^T \tilde \Pi \tilde \varepsilon_\theta \\
U^\theta_{\theta\theta} &= -{C^\theta}^{-1} \\
\end{align*}

where $Z^\theta$ is a normalisation constant. Here the precision matrix is updated from the prior precision. If we assume the parameters are not constant, but varying much more slowly than the states, then this could be estimated iteratively, assuming sufficient observations per update. In this case we would use:

\begin{align*}
\Delta \Pi^\theta &= \int \tilde \varepsilon_\theta^T \tilde \Pi \tilde \varepsilon_\theta \:dt \\
\end{align*}

Similarly, for the hyperparameters, but in this case it is the precision matrix $\tilde\Pi$ that is a function of the hyperparameters $\lambda$:

\begin{align*}
U(t) &= - \frac{N}{2} \ln 2\pi + \frac{1}{2} \ln |\tilde\Pi| -  \frac{1}{2}{{\tilde\varepsilon}^T \tilde\Pi \tilde\varepsilon} + \ln p(\tilde \nu) \\
\bar{V}^\lambda &= \int U(t) + W(t)^u + W(t)^\theta \:dt + U^\lambda \\
&= \int U(t) + W(t)^u + W(t)^\theta \:dt + {\varepsilon^\lambda}^T \Pi^\lambda \varepsilon^\lambda - Z^\lambda \\
\bar V_\lambda^\lambda &= \int \tilde \varepsilon_\lambda^T \tilde \Pi \tilde \varepsilon + W(t)_\lambda^u + W(t)_\lambda^\theta \:dt - \Pi^\lambda \varepsilon^\lambda \\
\bar V_{\lambda \lambda}^\lambda &= \int \tilde \varepsilon_\lambda^T \tilde \Pi \tilde \varepsilon_\lambda + W(t)_{\lambda \lambda}^u \:dt - \Pi^\lambda \\
W(t)_{\lambda_i}^u &= -\frac{1}{2} \text{tr} \left( \Sigma_i^u \tilde \varepsilon_{u}^{iT} Q_i \tilde \varepsilon_u^i \right) \\
W(t)_{\lambda_i}^\theta &= -\frac{1}{2} \text{tr} \left( \Sigma_i^\theta \tilde \varepsilon_{\theta}^{iT} Q_i \tilde \varepsilon_\theta^i \right) \\
W(t)_{\lambda \lambda}^u &= 0 \\
Q_i &= \frac{\partial \tilde\Pi}{\partial \lambda_i} \\
J(\lambda) &= \bar V_{\lambda \lambda}^\lambda \\
\Delta \mu^\lambda &= -J(\lambda)^{-1} \dot\mu^\lambda = - {\bar V_{\lambda \lambda}^\lambda}^{-1} \bar V_{\lambda}^\lambda\\
\Pi^\lambda &= -\bar{U}_{\lambda\lambda} = - \int U(t)_{\lambda\lambda} \: dt- U^\lambda_{\lambda\lambda} \\
U(t)_{\lambda \lambda} &= - \tilde \varepsilon_\lambda^T \tilde \Pi \tilde \varepsilon_\lambda \\
U^\lambda_{\lambda\lambda} &= -{C^\lambda}^{-1} \\
&\text{or}\\
\Delta \Pi^\lambda &= \int \tilde \varepsilon_\lambda^T \tilde \Pi \tilde \varepsilon_\lambda \:dt \\
\end{align*}

where $W(t)_{\lambda \lambda}^u = 0$ is because we assume that the system is linear in the hyperparameters. Although uncertainty about the hyperparameters does not affect the states and parameters, uncertainty about both the states and parameters affect the hyperparameter update. 


### Hierarchical dynamic model ###

For a hierarchical dynamic model (HDM), we assume each higher level generates causes for the level below, so that the causes $\nu$ link levels, whereas hidden states $x$ link dynamics over time. Further it is assumed that the noise processes at each level $w^{(i)}$ and $z^{(i)}$ are conditionally independent. This leads to the following Bayesian directed graph:

<img src="HDM.png" width="600">

Here $\vartheta^{(i)} = [\theta^{(i)}, \lambda^{(i)}]$ and $u^{(i)} = [\tilde \nu^{(i)}, \tilde x^{(i)}]$ and:

\begin{align*}
g_\nu &= 
\begin{bmatrix}
g_\nu^{(1)} & & \\
0 & \ddots & \\
& \ddots & g_\nu^{(m)} \\
& & 0 \\
\end{bmatrix}
&g_x = 
\begin{bmatrix}
g_x^{(1)} & & \\
0 & \ddots & \\
& \ddots & g_x^{(m)} \\
& & 0 \\
\end{bmatrix}\\
f_\nu &= 
\begin{bmatrix}
f_\nu^{(1)} & & \\
& \ddots & \\
& & f_\nu^{(m)} \\
\end{bmatrix}
&f_x = 
\begin{bmatrix}
f_x^{(1)} & & \\
& \ddots & \\
& & f_x^{(m)} \\
\end{bmatrix}\\
\end{align*}

Note that the partial derivatives of $g(x,\nu)$ have an extra row to accommodate the highest hierarchical level.

### Gradients and Curvature ###

For our HDM the gradients and curvature of the internal energy are:

\begin{align*}
U(t) &= \ln p(\vartheta, \tilde y, m) = - \frac{1}{2} \tilde \varepsilon^T \tilde \Pi \tilde \varepsilon \\
U(t)_u &= -\tilde \varepsilon_u^T \tilde \Pi \tilde \varepsilon \\
U(t)_\theta &= -\tilde \varepsilon_\theta^T \tilde \Pi \tilde \varepsilon \\
U(t)_{\lambda i} &= - \frac{1}{2} \text{tr} (Q_i(\tilde \varepsilon \tilde \varepsilon^T - \tilde \Sigma)) \\
U(t)_{uu} &= -\tilde \varepsilon_u^T \tilde \Pi \tilde \varepsilon_u \\
U(t)_{\theta\theta} &= -\tilde \varepsilon_\theta^T \tilde \Pi \tilde \varepsilon_\theta \\
U(t)_{\lambda\lambda ij} &= - \frac{1}{2} \text{tr} (Q_i \tilde \Sigma Q_j \tilde \Sigma)) \\
\end{align*}

with:

\begin{align*}
U(t)_{\lambda i} &= \frac{dU(t)}{d\lambda_i} \\
Q_{i} &= \frac{d\tilde\Pi}{d\lambda_i} \\
\end{align*}

and where we assume that the precision of the random fluctuations is linear in the hyperparameters:

\begin{align*}
\frac{d^2\tilde\Pi}{d\lambda_i^2} &= 0\\
\end{align*}

The derivatives with respect to each parameter $\tilde \varepsilon_{\theta}(t) = \tilde \varepsilon_{u\theta} \mu^u(t)$ rest on the second derivative of the models functions that mediate interactions between each parameter and the states:

\begin{align*}
\tilde \varepsilon_{u\theta} &= -
\begin{bmatrix}
I \otimes g_{v\theta} & I \otimes g_{x\theta} \\
I \otimes f_{v\theta} & I \otimes f_{x\theta}
\end{bmatrix}
\end{align*}